<img src="NotebookAddons/blackboard-banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="5"><b>Preparing a HyP3 InSAR Stack for MintPy</b></font>

<br>
<font size="4"><b> Alex Lewandowski; University of Alaska Fairbanks
    <br>
    Based off <a href="https://github.com/ASFHyP3/hyp3-docs" target="_blank">prep_hyp3_for_mintpy.ipynb </a> by Jiang Zhu; University of Alaska Fairbanks</b> <br>
<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" />
</font>
<br><br>

<font face="Calibri" size="5" color="darkred"> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>

In [ ]:

%%javascript
var kernel = Jupyter.notebook.kernel;
var command = ["notebookUrl = ",
               "'", window.location, "'" ].join('')
kernel.execute(command)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/insar_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "insar_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "insar_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "insar_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3">In this notebook we will use the following scientific libraries:
<ol type="1">
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>

</font>

<font face="Calibri" size="3"> Our first step is to <b>import them:</b> </font>

In [ ]:
from osgeo import gdal
from pathlib import Path
import pickle
import re

from osgeo import gdal
import pyproj

import asf_notebook as asfn

from hyp3_sdk import Batch, HyP3

%matplotlib notebook

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Your Own Data Stack Into the Notebook </b> </font>

<font size="3"> This notebook assumes that you've created an InSAR data stack over an area of interest using the <a href="https://www.asf.alaska.edu/" target="_blank">Alaska Satellite Facility's</a> value-added product system HyP3, available via <a href="https://search.asf.alaska.edu/#/" target="_blank">ASF Data Search (Vertex)</a>. HyP3 is an ASF service used to prototype value added products and provide them to users to collect feedback.

We will retrieve HyP3 data via the hyp3_sdk. As both HyP3 and the Notebook environment sit in the <a href="https://aws.amazon.com/" target="_blank">Amazon Web Services (AWS)</a> cloud, data transfer is quick and cost effective.</font>
</font>

<hr>
<font face="Calibri" size="3"> Before downloading anything, create an analysis directory to hold your data.
<br><br>
<b>Select or create a working directory for the analysis:</b></font>

In [ ]:
while True:
    data_dir = Path(asfn.input_path(f"\nPlease enter the name of a directory in which to store your data for this analysis."))
    if data_dir.is_dir():
        contents = data_dir.glob('*')
        if len(list(contents)) > 0:
            choice = asfn.handle_old_data(data_dir, list(contents))
            if choice == 1:
                shutil.rmtree(data_dir)
                data_dir.mkdir()
                break
            elif choice == 2:
                break
            else:
                clear_output()
                continue
        else:
            break
    else:
        data_dir.mkdir()
        break

<font face="Calibri" size="3"><b>Define absolute path to  analysis directory:</b></font>

In [ ]:
analysis_directory = Path.cwd().joinpath(data_dir)
print(f"analysis_directory: {analysis_directory}")

<font face="Calibri" size="3"><b>Create a HyP3 object and authenticate</b> </font>

In [ ]:
hyp3 = HyP3(prompt=True)

<font face="Calibri" size="3"><b>List projects containing active InSAR products and select one:</b></font>

In [ ]:
my_hyp3_info = hyp3.my_info()
active_projects = dict()

for project in my_hyp3_info['job_names']:
    batch = Batch()
    batch = hyp3.find_jobs(name=project, job_type='INSAR_GAMMA').filter_jobs(running=False, include_expired=False)
    if len(batch) > 0:
        active_projects.update({batch.jobs[0].name: batch})
        
if len(active_projects) > 0:
    display(Markdown("<text style='color:darkred;'>Note: After selecting a project, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:darkred;'>Otherwise, you will rerun this code cell.</text>"))
    print('\nSelect a Project:')
    project_select = asfn.select_parameter(active_projects)

project_select

<font face="Calibri" size="3"><b>Select a date range of products to download:</b> </font>

In [ ]:
jobs = project_select.value

display(Markdown("<text style='color:darkred;'>Note: After selecting a date range, you should select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you may simply rerun this code cell.</text>"))
print('\nSelect a Date Range:')
dates = asfn.get_job_dates(jobs)
date_picker = asfn.gui_date_picker(dates)
date_picker

<font face="Calibri" size="3"><b>Save the selected date range and remove products falling outside of it:</b> </font>

In [ ]:
date_range = asfn.get_slider_vals(date_picker)
date_range[0] = date_range[0].date()
date_range[1] = date_range[1].date()
print(f"Date Range: {str(date_range[0])} to {str(date_range[1])}")
jobs = asfn.filter_jobs_by_date(jobs, date_range)

<font face="Calibri" size="3"><b>Gather the available paths and orbit directions for the remaining products:</b></font>

In [ ]:
display(Markdown("<text style='color:darkred;'><text style='font-size:150%;'>This may take some time for projects containing many jobs...</text></text>"))
jobs = asfn.get_paths_orbits(jobs)
paths = set()
orbit_directions = set()
for p in jobs:
    paths.add(p.path)
    orbit_directions.add(p.orbit_direction)
paths.add('All Paths')
display(Markdown(f"<text style=color:blue><text style='font-size:175%;'>Done.</text></text>"))

<hr>
<font face="Calibri" size="3"><b>Select a path or paths (use shift or ctrl to select multiple paths):</b></font>

In [ ]:
display(Markdown("<text style='color:darkred;'>Note: After selecting a path, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Path:')
path_choice = asfn.select_mult_parameters(paths)
path_choice

<font face="Calibri" size="3"><b>Save the selected flight path/s:</b></font>

In [ ]:
flight_path = path_choice.value
if flight_path:
    if flight_path:
        print(f"Flight Path: {flight_path}")
    else:
        print('Flight Path: All Paths')
else:
    print("WARNING: You must select a flight path in the previous cell, then rerun this cell.")

<font face="Calibri" size="3"><b>Select an orbit direction:</b></font>

In [ ]:
if len(orbit_directions) > 1:
    display(Markdown("<text style='color:red;'>Note: After selecting a flight direction, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:red;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Flight Direction:')
direction_choice = asfn.select_parameter(orbit_directions, 'Direction:')
direction_choice

<font face="Calibri" size="3"><b>Save the selected orbit direction:</b></font>

In [ ]:
direction = direction_choice.value
print(f"Orbit Direction: {direction}")

<font face="Calibri" size="3"><b>Filter jobs by path and orbit direction:</b></font>

In [ ]:
jobs = asfn.filter_jobs_by_path(jobs, flight_path)
jobs = asfn.filter_jobs_by_orbit(jobs, direction)
print(f"There are {len(jobs)} products to download.")

<font face="Calibri" size="3"><b>Download the products, unzip them into a directory named after the product type, and delete the zip files:</b> </font>

In [ ]:
insar_path = analysis_directory.joinpath('INSAR_GAMMA')
if not insar_path.is_dir():
    insar_path.mkdir()

print(f"\nProject: {jobs.jobs[0].name}")
project_zips = jobs.download_files(insar_path)
for z in project_zips:
    asfn.asf_unzip(str(insar_path), str(z))
    z.unlink()

<font face="Calibri" size="3"><b>Subset the stack to the largest aoi covered by all input files and delete unneeded files:</b></font>

In [ ]:
fnames = list(insar_path.glob('*/*.tif'))
fnames.sort()

# determine the largest area covered by all input files
corners = [gdal.Info(str(f), format='json')['cornerCoordinates'] for f in fnames]
ulx = max(corner['upperLeft'][0] for corner in corners)
uly = min(corner['upperLeft'][1] for corner in corners)
lrx = min(corner['lowerRight'][0] for corner in corners)
lry = max(corner['lowerRight'][1] for corner in corners)

# clip the files
# regex = '(?<=_)(corr|unw_phase|water_mask|dem|lv_theta|inc_map|inc_map_ell|amp)(?=.tif)'
regex = '.tif'
for fname in fnames:
    if re.search(regex, str(fname)):
        fname_out = fname.parent.joinpath(Path(f"{fname.stem}_clip{fname.suffix}"))
        gdal.Translate(destName=str(fname_out), srcDS=str(fname), projWin=[ulx, uly, lrx, lry])
        fname.unlink()
        
# remove the *.xml, png, and *.kmz files
for pattern in ["xml","png","kmz","md.txt"]:
    unneeded_files = insar_path.glob(f"*/*.{pattern}")
    for file in unneeded_files:
        file.unlink()

**Subset the stack to a smaller AOI or skip this section and run the final code cell in the notebook for a link to the MintPy_Time_Series_From_Prepared_Data_Stack notebook**

In [ ]:
image_file = f"{analysis_directory}/raster_stack.vrt"
amp = list(insar_path.glob('*/*_amp_clip.tif'))[0]
!gdalbuildvrt -separate $image_file -overwrite $amp
img = gdal.Open(image_file)
rasterstack = img.ReadAsArray()

In [ ]:
fig_xsize = 7.5
fig_ysize = 7.5
aoi = asfn.AOI_Selector(rasterstack, fig_xsize, fig_ysize)

In [ ]:
geotrans = img.GetGeoTransform()

def geolocation(x, y, geotrans):
    return [geotrans[0]+x*geotrans[1], geotrans[3]+y*geotrans[5]]

In [ ]:
try:
    ul = geolocation(aoi.x1, aoi.y1, geotrans)
    lr = geolocation(aoi.x2, aoi.y2, geotrans)
#     print(f"aoi_coords in EPSG {utm}")
    print(f"upper left corner: {ul}")
    print(f"lower right corner: {lr}")
except TypeError:
    print('TypeError')
    display(Markdown(f'<text style=color:red>This error occurs if an AOI was not selected.</text>'))
    display(Markdown(f'<text style=color:red>Note that the square tool icon in the AOI selector menu is <b>NOT</b> the selection tool. It is the zoom tool.</text>'))
    display(Markdown(f'<text style=color:red>Read the tips above the AOI selector carefully.</text>'))

In [ ]:
clips = list(insar_path.glob('*/*clip.tif'))
clips.sort()

for i, fname in enumerate(clips):
    fname.rename(fnames[i])
    gdal.Translate(destName=str(fname), srcDS=str(fnames[i]), projWin=[ul[0], ul[1], lr[0], lr[1]])
    fnames[i].unlink() 

**Run the code cell below for a link to the MintPy_Time_Series_From_Prepared_Data_Stack notebook**

In [ ]:
display(Markdown(f'<text style=color:red> Open <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/notebooks/SAR_Training/English/Master/MintPy_Time_Series_From_Prepared_Data_Stack.ipynb" target="_blank"> MintPy_Time_Series_From_Prepared_Data_Stack.ipynb </a>to run a time series analysis on your data using MintPy</text>'))

<font face="Calibri" size="2"> <i>Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb - Version 1.1.0 - September 2021
    <ul>
        <li>Add AOI subsetting</li>
        <li>Link to MintPy_Time_Series_From_Prepared_Data_Stack.ipynb</li>
    </ul>
    </i>
</font>